<a href="https://colab.research.google.com/github/0sparsh2/Indigo-Twitter-Analysis/blob/main/NLP_Baggage_ai_Sparsh_Nagpal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Assignment 1

Scrape 500 Tweets that has tagged Indigo airlines. Scrape the tweet, ID of the
tweet, reply, timestamp, name and location of the user as shown in figure 1.
Include "Replies" ONLY BY Indigo, if not then input 0.
Make sure you filter out "Retweets".
(You can use Twitter API)

In [ ]:
!pip install tweepy==4.4.0

  Using cached tweepy-4.4.0-py2.py3-none-any.whl (65 kB)
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0


In [ ]:
#importing libraries
import pandas as pd
import tweepy
import time
import warnings
warnings.filterwarnings('ignore')

###Credentials

In [ ]:
#Removing for privacy reasons
consumer_key = xxxxxxxxxxxxxxxxxxxxx
consumer_secret = xxxxxxxxxxxxxxxxxxxxx
access_token = xxxxxxxxxxxxxxxxxxxxx
access_token_secret = xxxxxxxxxxxxxxxxxxxxx

In [ ]:
#Accessing twitter API
auth=tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token=(access_token, access_token_secret)
api= tweepy.API(auth, wait_on_rate_limit= True)

In [ ]:
client2 = tweepy.Client(consumer_key= consumer_key,consumer_secret= consumer_secret,access_token= access_token,access_token_secret= access_token_secret ,wait_on_rate_limit=True)

In [ ]:
client = tweepy.Client(xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx)
#Hiding for privacy
pd.set_option('display.max_colwidth', -1)

###Code

In [ ]:
tweets = []
tweet_id = []
name = []
reply = []
timestamp = []
location = []

#indigo6e id = 1467228402
for tweet in tweepy.Paginator(client.get_users_mentions, 1467228402,  tweet_fields=["created_at", "lang"], place_fields=['country,full_name,id,name,place_type']).flatten(limit=500):
    tweets.append(tweet.text)
    #print('Tweet text: ',tweet.text)
    c2 = client2.get_tweet(id= tweet.id, expansions="geo.place_id,entities.mentions.username,in_reply_to_user_id,referenced_tweets.id.author_id" ,user_auth=True)

    #print('Tweet ID',tweet.id)
    tweet_id.append(tweet.id)
    #print('Timestamp: ',tweet.created_at)
    timestamp.append(tweet.created_at)
    try:
      name.append(c2.includes['users'][0].name)
      #print('Users: ',c2.includes['users'][0].name)
    except:
      pass

    try:
      location.append(c2.includes['places'])
      #print('Place: ',c2.includes['places'])
    except:
      location.append(None)
      pass

    try:
      #print('Replies: ',c2.includes['tweets'])
      #print('reply id: ',c2.includes['tweets'][0].id)
      #print('reply text: ',c2.includes['tweets'][0].text )
      #print('User replying: ', (client2.get_tweet(c2.includes['tweets'][0].id, expansions='referenced_tweets.id.author_id,in_reply_to_user_id', user_auth=True )).includes['users'][0])
      
      user_replying = client2.get_tweet(c2.includes['tweets'][0].id, expansions='referenced_tweets.id.author_id,in_reply_to_user_id', user_auth=True ).includes['users'][0]
      if len(c2.includes['tweets'])>0:
        if user_replying =='IndiGo6E':
          reply.append(c2.includes['tweets'][0].text)
        else:
          reply.append(None)
    except:
      reply.append(None)
      pass

In [ ]:
import numpy as np
tweet_list = [tweets, tweet_id, reply, timestamp, name, location]
print([len(i) for i in tweet_list])

[500, 500, 500, 500, 500, 500]


In [ ]:
tweets_set = [[tweets[i],tweet_id[i], reply[i], timestamp[i], name[i], location[i]] for i in range(len(tweets))]

In [ ]:
tweets_indigo = pd.DataFrame(tweets_set, columns=['Tweet','tweet_id','Replies','Timestamp','Name','Location'])

###Output Dataframe

In [ ]:
tweets_indigo.head()

,Tweet,tweet_id,Replies,Timestamp,Name,Location
0,@LiveFromALounge @IndiGo6E Actually prefer the cucumber tomato over the junglee! Goes better with chips,1486653884999241728,None,2022-01-27 10:54:41+00:00,Rahul,None
1,"@IndiGo6E PNR sent. you can check at your end (don't change anything pls, just let me know if available or not)",1486653781689262080,None,2022-01-27 10:54:16+00:00,Ajay Awtaney,None
2,@SK10940154 @LiveFromALounge @IndiGo6E I felt the need to have it on GAU BOM 3.5 hr long morning flight,1486653547575869442,None,2022-01-27 10:53:21+00:00,Abhishek Biswas,None
3,@IndiGo6E my flight was cancelled by indigo so I select for refund process. I talked with customer care person and she told me I will get full refund but I got only 10% refund mail.,1486652155721228291,None,2022-01-27 10:47:49+00:00,Ishani,None
4,"@LiveFromALounge @IndiGo6E Oops, forgot to see the price. Free is always welcome.",1486651977404616704,None,2022-01-27 10:47:06+00:00,SK,None


TweepError: ignored

##Assignment 2

Entity Extraction:
Extract PNR number and the tags in the tweets as shown in figure 3. Add these
2 columns in the data frame as separate columns.

In [ ]:
def tag_detect(string):
  tags = [i for i in string.split() if i[0]=='@']
  tags_joined = " ".join(tags)
  return tags_joined

In [ ]:
tweets_tags = [tag_detect(tweet) for tweet in tweets]

In [ ]:
len(tweets_tags)

500

In [ ]:
import re
pnr_tweets = []
for tweet in tweets:
  tweet = re.sub('[^0-9A-Za-z]',' ',tweet)
  pnrs = " ".join([pnr for pnr in tweet.split() if len(pnr)==6 and len(re.findall('\D',pnr))!=6 and pnr.isupper()])
  if len(pnrs)!=0:
    pnr_tweets.append(pnrs)
    #print(pnrs)
  else:
    pnr_tweets.append(None)
#pnr_tweets

In [ ]:
pnr_lists = [[tweets[i], pnr_tweets[i], tweets_tags[i]] for i in range(500)]

In [ ]:
pnr_df = pd.DataFrame(pnr_lists, columns=["Tweet", "PNR", "Tags"])

In [ ]:
pnr_df.head(100)

,Tweet,PNR,Tags
0,@LiveFromALounge @IndiGo6E Actually prefer the cucumber tomato over the junglee! Goes better with chips,None,@LiveFromALounge @IndiGo6E
1,"@IndiGo6E PNR sent. you can check at your end (don't change anything pls, just let me know if available or not)",None,@IndiGo6E
2,@SK10940154 @LiveFromALounge @IndiGo6E I felt the need to have it on GAU BOM 3.5 hr long morning flight,None,@SK10940154 @LiveFromALounge @IndiGo6E
3,@IndiGo6E my flight was cancelled by indigo so I select for refund process. I talked with customer care person and she told me I will get full refund but I got only 10% refund mail.,None,@IndiGo6E
4,"@LiveFromALounge @IndiGo6E Oops, forgot to see the price. Free is always welcome.",None,@LiveFromALounge @IndiGo6E
...,...,...,...
95,@happyeasygo @gurgaonpolice @DGCAIndia @IndiGo6E https://t.co/s3jqM48MtG,None,@happyeasygo @gurgaonpolice @DGCAIndia @IndiGo6E
96,@IndiGo6E You have not understood my #concern.I got a damaged bag aftr my travel in @IndiGo6E airlines. I want @IndiGo6E to pay me for replacing this bag.I can't use it https://t.co/PQcFMyJ1Sg can't deny your responsibility towards customers.I'll resort to consumer forum and legal action,None,@IndiGo6E @IndiGo6E @IndiGo6E
97,@IndiGo6E I can't able to open the link that you people provided in the mail inorder to edit my flight,None,@IndiGo6E
98,@anmol_sikka @HardeepSPuri @IndiGo6E @DelhiAirport @bengaluruairprt Same is like flipkart flight had done with me. Scam,None,@anmol_sikka @HardeepSPuri @IndiGo6E @DelhiAirport @bengaluruairprt


##Assignment 3

Identify the context of the tweet as shown in figure 2. Add the context in the
data frame as a separate column.

##The assignment is present in the other colab notebook

In [ ]:
tweets

['@LiveFromALounge @IndiGo6E Actually prefer the cucumber tomato over the junglee! Goes better with chips',
 "@IndiGo6E PNR sent. you can check at your end (don't change anything pls, just let me know if available or not)",
 '@SK10940154 @LiveFromALounge @IndiGo6E I felt the need to have it on GAU BOM 3.5 hr long morning flight',
 '@IndiGo6E my flight was cancelled by indigo so I select for refund process. I talked with customer care person and she told me I will get full refund but I got only 10% refund mail.',
 '@LiveFromALounge @IndiGo6E Oops, forgot to see the price. Free is always welcome.',
 '@SK10940154 @IndiGo6E You don’t have to eat it on board, you can bring it home with you.',
 '@LiveFromALounge @IndiGo6E A better question would be who is brave enough to take off their mask in a tight space where people have been packed like sheep?',
 'Is there a glitch in the matrix? On an upcoming @IndiGo6E flight there is no Chicken Junglee Sandwich but a new Chicken Cucumber Tomato Sandw